In [2]:
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from zipfile import ZipFile
import shutil
import glob

In [3]:
project_path = r"..\Datasets_test_binary\OneDrive_2022-09-22\Dataset projet"
other_dir = project_path + r"\other"
photo_dir = project_path + r"\photo"

In [4]:
train_set, test_set = tf.keras.preprocessing.image_dataset_from_directory(
    project_path,
    validation_split=0.2,
    seed=10,
    image_size = (384, 384),
    batch_size=128,
    subset = "both"
)
print(f"\nThe dataset is composed of these classes : {train_set.class_names}")


Found 41405 files belonging to 2 classes.
Using 33124 files for training.
Using 8281 files for validation.

The dataset is composed of these classes : ['Other', 'Photo']


# III. Mise en place des modèles

Expliquer la mise en place d'un modèle : 

- Definition du modèle
- Compilation du modèle (expliquer optimizer et function loss // pourquoi on a choisi ceux-là)
- Fit du modèle

## A. Modèles de base

### 1. NN

#### 1.1. Définition du modèle

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255, offset=0.0),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#### 1.2. Compilation du modèle

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.losses.BinaryCrossentropy(),
    metrics=['accuracy'])

model.summary()

#### 1.3. Entrainement du modèle

In [ ]:
epochs = 10

history = model.fit(train_set, validation_data=test_set, epochs=epochs)

#### 1.4. Résultats du modèle

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### 2. CNN

#### 2.1. Définition du modèle

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255, offset=0.0),
    tf.keras.layers.Conv2D(16, 3, padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#### 2.2. Compilation du modèle

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.losses.BinaryCrossentropy(),
    metrics=['accuracy'])

model.summary()

#### 2.3. Entrainement du modèle

In [ ]:
epochs = 10

history = model.fit(train_set, validation_data=test_set, epochs=epochs)

#### 2.4. Résultats du modèle

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## B. Régularisation

La régularisation en intelligence artificielle est une technique utilisée pour empêcher ou réduire le surapprentissage d'un modèle de deep learning. La régularisation consiste à ajouter une contrainte au processus d'apprentissage du modèle afin de le forcer à rester simple et généralisable. En utilisant cette technique, on peut améliorer les performances d'un modèle sur de nouvelles données et le rendre plus capable de gérer des situations inattendues.

Il existe plusieurs méthodes de régularisations comme par exemple :


- le Dropout : cette méthode consiste à désactiver aléatoirement un pourcentage de neurones d'une couche pendant l'entraînement. Cela permet d'empêcher les neurones de dépendre trop les uns des autres et d'améliorer la généralisation de la modèle à de nouvelles données.


- la Data Augmentation : cette méthode consiste à générer de nouvelles données à partir des données d'entraînement existantes en appliquant des transformations aléatoires telles que des rotations, des translations, des miroirs, des changements de luminosité, etc... dans le but d'augmenter la quantité de données disponibles pour l'entraînement d'un modèle.


- la régularisation L1, L2, Elastic-Net : ces méthodes consistent à modifier la fonction de coût dans le but  de réduire la complexité du modèle en mettant à zéro certains poids (L1), en atténuant les valeurs des poids (L2), ou en combinant ces deux actions (Elastic-Net). 


- la Batch Normalization : cette méthode consiste à normaliser les activations d'une couche en utilisant les moyennes et les écarts-types des activations de chaque batch d'exemples d'entraînement. Cela a pour effet d'améliorer la convergence et la stabilité des réseaux de neurones.


- l'Early Stopping : cette méthode consiste à arrêter l'entraînement d'un modèle avant que la performance ne commence à décliner sur l'ensemble de validation.



### 1. Dropout

Ici, nous allons utiliser un dropout à 20%. Cela signifie que sur toutes les couches contraintes avec un dropout, 20% des neurones seront aléatoirement désactivés par itération.

#### 1.1. Définition du modèle

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255, offset=0.0),
    tf.keras.layers.Conv2D(16, 3, padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#### 1.2. Compilation du modèle

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.losses.BinaryCrossentropy(),
    metrics=['accuracy'])

model.summary()

#### 1.3. Entrainement du modèle

In [ ]:
epochs = 10

history = model.fit(train_set, validation_data=test_set, epochs=epochs)

#### 1.4. Résultats du modèle

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### 2. Résultats des autres régularisations testées

Tableau récapitulatif des résultats des autres régularisations testées

| Nom | Description | Précision d'entrainement| Précision de test | Nombre d'époque | Temps moyen par époque|
| :-------: |:---------------:|:-----:|:---------:|:---------:|:---------:|
|CNN (WS) + Dropout 20% + Padding|Modèle seul|0.93|0.93|10|2min|
|CNN (WS) + Dropout 20% + Padding|Modele seul + L1L2 Regularization (Elastic-Net)|0.79|0.70|10|1min45|
|CNN (WS) + Dropout 20% + Padding|Modèle seul + Batch Normalization|0.96|0.90|10|2min|
|CNN (WS) + Dropout 20%|Modèle + Data Augmentation|0.89|0.87|10|8-10min|
|CNN (WS) + Dropout 20%|Modèle seul|0.98|0.90|20|2min|
|CNN (WS) + Dropout 20%|Modèle seul + rotation des images dans un même sens (taille)|0.95|0.89|10|2min|
|CNN (WS) + Dropout 20%|Modèle seul + rotation des images dans un même sens + data augmentation|0.87|0.83|10|8min30|
|CNN (WS) + Dropout 20%|Modèle seul + DataGenerator|0.87|0.84|10|3min15|
|CNN (WS) + Dropout 20%|Modèle seul + DataGenerator + DataAugmentation|0.87|0.85|20|5min|
|CNN (WS) + Dropout 50%|Modèle seul|0.88|0.90|20|2min|
|CNN (WS) + Dropout 35%|Modèle seul|0.93|0.89|15|2min|
|CNN (WS) + Dropout 50%|Modèle seul|0.91|0.87|10|2min|
|CNN|Modèle + data augmentation|0.90|0.86|10|6min40|
|CNN|Modèle seul|0.99|0.91|10|1min50|
|CNN|Modèle seul + rotation des images dans un même sens (taille)|0.99|0.91|10|1min40|
|CNN|Modèle seul + rotation des images dans un même sens + data augmentation|0.90|0.86|10|7-8min|
|Pas de CNN (WS)|Modèle seul (Pas de CNN)|0.79|0.76|10|1min20|

## C. Architecture existantes

Enfin, nous avons voulu utiliser des architechtures déjà existants, dans la litterature scientifique. Pour cela, nous avons d'abord choisi de reproduire une architecture existante. Pour cela, nous avons cherché des modèles performants et comprenant relativement peu de couches, afin de pouvoir l'executer sur nos machines.

Ainsi, nous avons utilisé le modèle VGG11 qui est un bon compromis de nos contraintes.

### 1. VGG11

Le modèle VGG11 a été développé par l'équipe de recherche Visual Geometry Group (VGG) de l'université de Oxford en 2014. Il a été présenté pour la première fois dans un article de recherche intitulé "Very Deep Convolutional Networks for Large-Scale Image Recognition". Depuis sa publication, le modèle VGG11 est devenu l'un des modèles de reconnaissance d'images les plus populaires et a été utilisé dans de nombreuses recherches et applications en vision par ordinateur. Il est souvent utilisé comme base pour le développement de nouvelles méthodes en reconnaissance d'images et reste l'un des modèles les plus performants dans de nombreuses tâches de vision par ordinateur.

Ce réseau de neurones a été entraîné sur la base de données ImageNet. Il utilise une architecture de réseau convolutionnel profond avec 11 couches de convolution et 3 couches fully connected.

| Nom | Description | Précision d'entrainement| Précision de test | Nombre d'époque | Temps moyen par époque|
| :-------: |:---------------:|:-----:|:---------:|:---------:|:---------:|
|VGG-11 architecture|Modèle seul|0.95|0.91|20|3-4min|
|VGG-11 architecture customisée|Modèle seul|0.92|0.91|10|1min30|

### 2. EfficientNetV2B0 en Transfert Learning

EfficientNetV2B0 est un modèle de reconnaissance d'images pré-entraîné développé par Google et publié en 2021, qui utilise une architecture de réseau de neurones convolutionnel (CNN) optimisée pour être efficace en termes de consommation de ressources informatiques tout en conservant une haute performance. Il a été entraîné sur la base de données ImageNet et a été utilisé avec succès dans de nombreuses tâches de vision par ordinateur, telles que la détection d'objets, la classification d'images et la génération de descriptions de contenu d'image. Ce réseau de neurones a été entraîné sur la base de données ImageNet.

Le transfert learning est une technique qui permet de réutiliser les connaissances acquises par un modèle pré-entraîné sur une tâche donnée pour entraîner un nouveau modèle sur une tâche différente. Cela peut être utile lorsqu'il y a peu de données disponibles pour entraîner un modèle sur la tâche spécifique, ou lorsqu'il est difficile de collecter ces données. Le transfert learning peut également aider à améliorer les performances d'un modèle sur une tâche spécifique en utilisant les connaissances acquises par un modèle pré-entraîné sur une tâche similaire.

#### 2.1. Définition du modèle

In [ ]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow import keras
from keras import layers

class OwnEfficientNetV2B0:  
    @staticmethod
    def build(input_shape, trainable=False, dropout=0.2):
        inputs = keras.Input(shape=input_shape)

        baseModel = EfficientNetV2B0(weights="imagenet", include_top=False, input_tensor=inputs)
        baseModel.trainable = trainable

        headModel = baseModel.output
        headModel = layers.GlobalAveragePooling2D()(headModel)
        headModel = layers.Dropout(dropout)(headModel)
        outputs = layers.Dense(1, activation="sigmoid")(headModel)
        model = tf.keras.Model(inputs, outputs)

        return model

model = OwnEfficientNetV2B0().build(input_shape=(384, 384, 3))

#### 2.2. Compilation du modèle

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.losses.BinaryCrossentropy(),
    metrics=['accuracy'])

model.summary()

#### 2.3. Entrainement du modèle

In [ ]:
epochs = 10

history = model.fit(train_set, validation_data=test_set, epochs=epochs)

#### 2.4. Résultats du modèle

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# IV. Résultats et conclusion

| Nom | Description | Précision d'entrainement| Précision de test | Nombre d'époque | Temps moyen par époque|
| :-------: |:---------------:|:-----:|:---------:|:---------:|:---------:|
|CNN (WS) + Dropout 20% + Padding|Modèle seul|0.93|0.93|10|2min|
|CNN (WS) + Dropout 20% + Padding|Modele seul + L1L2 Regularization (Elastic-Net)|0.79|0.70|10|1min45|
|CNN (WS) + Dropout 20% + Padding|Modèle seul + Batch Normalization|0.96|0.90|10|2min|
|CNN (WS) + Dropout 20%|Modèle + Data Augmentation|0.89|0.87|10|8-10min|
|CNN (WS) + Dropout 20%|Modèle seul|0.98|0.90|20|2min|
|CNN (WS) + Dropout 20%|Modèle seul + rotation des images dans un même sens (taille)|0.95|0.89|10|2min|
|CNN (WS) + Dropout 20%|Modèle seul + rotation des images dans un même sens + data augmentation|0.87|0.83|10|8min30|
|CNN (WS) + Dropout 20%|Modèle seul + DataGenerator|0.87|0.84|10|3min15|
|CNN (WS) + Dropout 20%|Modèle seul + DataGenerator + DataAugmentation|0.87|0.85|20|5min|
|CNN (WS) + Dropout 50%|Modèle seul|0.88|0.90|20|2min|
|CNN (WS) + Dropout 35%|Modèle seul|0.93|0.89|15|2min|
|CNN (WS) + Dropout 50%|Modèle seul|0.91|0.87|10|2min|
|VGG-11 architecture|Modèle seul|0.95|0.91|20|3-4min|
|VGG-11 architecture customisée|Modèle seul|0.92|0.91|10|1min30|
|CNN|Modèle + data augmentation|0.90|0.86|10|6min40|
|CNN|Modèle seul|0.99|0.91|10|1min50|
|CNN|Modèle seul + rotation des images dans un même sens (taille)|0.99|0.91|10|1min40|
|CNN|Modèle seul + rotation des images dans un même sens + data augmentation|0.90|0.86|10|7-8min|
|Pas de CNN (WS)|Modèle seul (Pas de CNN)|0.79|0.76|10|1min20|
|EfficientNetV2B0|Modèle EfficientNetV2B0 (gelé) Modification des deux derniere couches : GlobalAveragePooling2D Dense avec 1 neurone en sigmoid|0.99|0.99|10|2min|